# 1. Frame the problem
#### Create an AI that will be able to play the clssic puzzle game "tetris", surviving for as long as possible.
- The game has a 10 block long, 20 block high board
- There are 6 different pieces, each one being composed of 4 blocks: line, z, flipped z, square, L, flipped L
- You are able to rotate all pieces to any one of 4 orientations
- The goal of the game is to clear as many lines as possible without "topping out", meaning that your blocks eventually stack over a certain height limit, causing you to lose.
- You are given a piece completely randomly every time, meaning that being given a fatal set of pieces is possible but rare (the worst possible set of pieces is actually surviveable until about 30 lines are cleared assuming you play perfectly)
- Some other modes of the game exist such as a timed 40 lines mode, but for the purposes of this model we are using the "endless" mode, with the goal of surviving as long as possible and clearing as many lines as possible

# 2. Get the Data
- This step is very different in this case compared to previous projects due to the nature of the model, which is meant to train on a zero sum game that runs in real time and has no static data or consistent pattern to follow due to the randomness of the pieces
- Since this model is meant to play tetris in real time and does not have any true "database" to train on, "getting" the data involves playing the game multiple times and getting the data of each game, so basically reinforcing your model in real time.
#### How the model actually plays the game
- I was provided with an already made tetris game framework, so that I wouldn't have to code an entire tetris game myself.
- As opposed to loading some data from a file or source, we are instead providing the model with information about the instantaneous state of a tetris game at all times. (By state I mean like "piece" or "turn")
- The model isn't going to make decisions based off the results of an entire game, but instead going to treat each and every single piece as its own seperate case with no regard for the previous or next piece.
- The specific information we give the model to make an instantaneous decision is just an array depicting the piece it will use and the current state of the board
#### How the model is trained
- Although the model only considers the board at an instantaneous state, the actual training of the model requires the result of several different games, so sort of a seperate data

# 3. Explore the Data
- Since there is no "database" of data, there is no concrete way to explore the data we will be feeding the algorithm, although it is possible to gain familiarity with the game in 2 ways:
#### Playing the game
- By playing the game of tetris, whether it be using the player mode of the given tetris framework or maybe using a more refined version on the web, you begin to understand what moves are best, which can help you conceptualize what features to give a genetic algorithm when it comes to how the algorithm will make decisions based on weights.
#### Exploring the given repository
- Although this isn't really exploring the data of the game itself, it is still important to get familiar with the framework to understand how to utilize it.
- The frameworks comes with several pre-made models that you can test and also use as a base for your own implementation of a model. These models are:
    - A genetic algorithm (that does very poorly but provides useful insight to how a genetic algorithm is meant to be implemented and can be used as a reference for our genetic algorithm that will be implemented)
    - A random algorithm (that understandably does very poorly)
    - A Monte Carlo tree search algorithm (that does very poorly)
    - A greedy algorithm (that does really well but isn't really considered a machine learning algorithm, meaning we can't make our own)
- After having tested these algorithms, it is also useful to explore the structure of the framework. There is a board.py that handles the state of the entire tetris board in a board object, a piece.py that handles a piece object for the currently falling piece in the game at any time, and a game.py that puts everything together, handling how different models are run and the overall state of the game.
- When game.py is run in AI mode, it calls the AI's "get_best_move" method which is meant to return an X value and a piece (which is just an array of the piece that was passed in, but rotated to the desired orientation)
- This "get_best_move" method is where all the actual decision making happens

# 4. Prepare the Data
- There is no data preparation to be done, since the data is not concrete and in a database. The instantaneous board data is already given in an organized format to the AI each time it is referenced.

# 5. Model the data
- After having read the paper about tetris AI algorithms included with the tetris game framework and also researching for several hours online about the mathematical/AI implications of tetris, I decided that my best course of action would be to create a genetic algorithm that makes decisions based on weights for each feature
- 

In [ ]:
import numpy as np
from game import Game
from genetic import Genetic_AI
import random
import pandas as pd


def cross(a1, a2, aggregate="lin"):
    """
    Compute crossover of two agents, returning a new agent
    """
    new_genotype = []
    a1_prop = a1.fit_rel / a2.fit_rel
    for i in range(len(a1.genotype)):
        rand = random.uniform(0, 1)
        if rand > a1_prop:
            new_genotype.append(a1.genotype[i])
        else:
            new_genotype.append(a2.genotype[i])

    return Genetic_AI_Victor(genotype=np.array(new_genotype), aggregate=aggregate, mutate=True)


def compute_fitness(agent, num_trials):
    """
    Given an agent and a number of trials, computes fitness as
    arithmetic mean of lines cleared over the trials
    """
    fitness = []
    for _ in range(num_trials):
        game = Game('genetic', agent=agent)
        peices_dropped, rows_cleared = game.run_no_visual()
        fitness.append(peices_dropped)
        print(f"    Trial: {_}/{num_trials}")

    # NOTE: consider dropping outliers for smoother performance
    return np.average(np.array(fitness))



def run_X_epochs(num_epochs=10, num_trials=5, pop_size=100, aggregate='lin', num_elite=5, survival_rate=.35, logging_file='default.csv'):

    # data collection over epochs
    data=[[1, np.ones(9), 1, np.ones(9),  1, np.ones(9)]]
    headers = ['avg_fit','avg_gene', 'top_fit', 'top_gene', 'elite_fit', 'elite_gene']
    df = pd.DataFrame(data, columns=headers)
    df.to_csv(f'data/{logging_file}.csv', index=False)

    # create inital population
    population = [Genetic_AI(aggregate=aggregate) for _ in range(pop_size)]

    for epoch in range(num_epochs):
        """
        Fitness
        """

        # data collection within epochs
        total_fitness = 0
        top_agent = 0
        gene = np.zeros(9)

        for n in range(pop_size):
            # compute fitness, add to total
            print(f"Agent: {n}/{pop_size}")
            agent = population[n]
            agent.fit_score = compute_fitness(agent, num_trials=num_trials)
            total_fitness += agent.fit_score 
            gene+=agent.genotype


        # compute % of fitness accounted for by each agent
        for agent in population:
            agent.fit_rel = agent.fit_score / total_fitness

        """
        Selection
        """

        next_gen = []

        # sort population by descending fitness
        sorted_pop = sorted(population, reverse=True)

        # elite selection: copy over genotypes from top preforming agents
        elite_fit_score = 0
        elite_genes = np.zeros(9)
        top_agent=sorted_pop[0]

        for i in range(num_elite):
            elite_fit_score +=sorted_pop[i].fit_score
            elite_genes += sorted_pop[i].genotype
            next_gen.append(Genetic_AI(genotype=sorted_pop[i].genotype, mutate=False))

        # selection: select top agents as parents base on survival rate
        num_parents = round(pop_size * survival_rate)
        parents = sorted_pop[:num_parents]

        # crossover: randomly select 2 parents and cross genotypes
        for _ in range(pop_size-(num_elite)):
            # randomly select parents, apply crossover, and add to the next generation
            # the cross functions automatically applies mutation to the new agent
            parents = random.sample(parents, 2)
            next_gen.append(cross(parents[0], parents[1], aggregate=aggregate))


        avg_fit = (total_fitness/pop_size)
        avg_gene = (gene/pop_size)
        top_fit = (top_agent.fit_score)
        top_gene = (top_agent.genotype)
        elite_fit = (elite_fit_score/num_elite)
        elite_gene = (elite_genes/num_elite)

        data = [[avg_fit, avg_gene, top_fit, top_gene, elite_fit, elite_gene]]
        df = pd.DataFrame(data, columns=headers)
        df.to_csv(f'data/{logging_file}.csv', mode='a', index=False, header=False)


        print(f'\nEpoch {epoch}: \n    total fitness: {total_fitness/pop_size}\n    best agent: {top_agent.fit_score}\n')

        population = next_gen

    return data


        


if __name__ == '__main__':
    run_X_epochs(num_epochs=15, num_trials=5, pop_size=50, num_elite=5)


In [ ]:
from tetris.genetic_victor import Genetic_AI_Victor
from tetris.game import Game

# from multiprocessing import Pool
import random
import numpy as np
import pickle


pSize = 100
genLimit = 100

population = []
for i in range(pSize):
    population.append(Genetic_AI_Victor())

def run(agent):
    game = Game("genetic_victor", agent)
    dropped, cleared = game.run_no_visual()
    #print(f'Dropped: {dropped}, Cleared: {cleared}')
    return cleared

def breed(agent1, agent2):
    #it is assumed all agents will have the same amount of features
    
    # Ensure crossover_point is within the bounds of the shorter genotype
    crossover_point = random.randint(0, max_len)
    
    # Perform crossover with padded genotypes
    child_genotype = np.concatenate((agent1[:crossover_point], agent2[crossover_point:]))
    
    child = Genetic_AI_Victor(genotype = child_genotype)
    return child


def compute_fitness(agent, num_trials):
    """
    Given an agent and a number of trials, computes fitness as
    arithmetic mean of lines cleared over the trials
    """
    fitness = []
    for _ in range(num_trials):
        game = Game('genetic', agent=agent)
        peices_dropped, rows_cleared = game.run_no_visual()
        fitness.append(peices_dropped)
        print(f"    Trial: {_}/{num_trials}")

    # NOTE: consider dropping outliers for smoother performance
    return np.average(np.array(fitness))
    
with Pool(30) as pool:
    for gen in range(genLimit):
        scoreList = pool.map(runGame, population)
        listedScore = [(score, i) for i, score in enumerate(scoreList)]
        sortedScores = sorted(listedScore, key=lambda x: x[0], reverse=True)

        liversList = [i for _, i in sortedScores[:int(0.3*populationSize)]]
        #print(liversList)
        livers = [population[i] for i in liversList]

        genBest, childID = sortedScores[0][0], sortedScores[0][1]

        if genBest > best:
            best = genBest
            genNum = gen+1
            bestChildID = childID
            currentBest = population[int(childID)]
            currentBestScore = genBest

        if (gen+1)%1 == 0:
            print('.', end='')
            best = 0
            spacer = str(genLimit)[len(str(genNum+1)):]
            joblib.dump(currentBest, f'v1/{valVal}/v1g{spacer}{genNum}.joblib')
            with open(f'v1/{valVal}/README.txt', 'a') as f:
                f.write(f'Gen: {genNum}, Child {bestChildID} achieved score {currentBestScore}\n')
        if (gen+1)%1000 == 0:
            print(f']\nGen: {gen+1}, Child {childID} achieved score {genBest}\n[')
            #joblib.dump(population[int(childID)], f'v1/v1m{gen+1}.joblib')
            
        
        newPopulation = []
        _ = 0
        while _ < populationSize:
            parent1 = random.choice(livers)
            parent2 = random.choice(livers)
            child = breed(parent1, parent2)
            if child is not None:
                newPopulation.append(child)
                _ += 1

        population = newPopulation

    import joblib
    #joblib.dump(population[int(childID)], 'v1/v1m1.joblib')

print('Done')